<div class="alert alert-block alert-info">
<b> Keywords: FEATURE ENG - Encoding categorical variables</b></div>
    
    - Replace values or mapping -> manual encoding, ie Yes: 1; No:0
    - Label Encoding -> categorical vars that have a hierarchy, order
    - Ordinal Encoding -> Label Encoding with more than one var in one line
    - One Hot Encoding -> used for nominal variables (no hierarchy). But watch out with # categories in it!
    - Binary encoding -> nominal variables when the #of categories is too high

# Enconding of Categorical Variables

In this notebook, you'll focus on dealing with categorical features in the `pnwflights14 dataset`, but you can apply the same procedure to all kinds of datasets. pnwflights14 contains information about all flights that departed from the two major airports of the Pacific Northwest (PNW), SEA in Seattle and PDX in Portland, in 2014: 162,049 flights in total.



# Loading the data and some EDA <a class="anchor" id="1"></a>

[Index](#0.1)

In [4]:
import pandas as pd
import numpy as np
import copy
import warnings


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
from google.colab import files
uploaded = files.upload()


Saving flight (1).csv to flight (1).csv


In [7]:
df_flights = pd.read_csv('flight (1).csv', index_col = 0)

In [8]:

df_flights


,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2014,1,1,1.0,96.0,235.0,70.0,AS,N508AS,145,PDX,ANC,194.0,1542,0.0,1.0
1,2014,1,1,4.0,-6.0,738.0,-23.0,US,N195UW,1830,SEA,CLT,252.0,2279,0.0,4.0
2,2014,1,1,8.0,13.0,548.0,-4.0,UA,N37422,1609,PDX,IAH,201.0,1825,0.0,8.0
3,2014,1,1,28.0,-2.0,800.0,-23.0,US,N547UW,466,PDX,CLT,251.0,2282,0.0,28.0
4,2014,1,1,34.0,44.0,325.0,43.0,AS,N762AS,121,SEA,ANC,201.0,1448,0.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162044,2014,9,30,2357.0,-2.0,545.0,-20.0,UA,N477UA,479,PDX,IAH,210.0,1825,23.0,57.0
162045,2014,9,30,2359.0,4.0,734.0,-1.0,DL,N137DL,1929,SEA,ATL,244.0,2182,23.0,59.0
162046,2014,9,30,NaN,NaN,NaN,NaN,AS,N530AS,470,SEA,SAN,NaN,1050,NaN,NaN
162047,2014,9,30,NaN,NaN,NaN,NaN,US,NaN,494,PDX,PHX,NaN,1009,NaN,NaN


In [9]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162049 entries, 0 to 162048
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   year       162049 non-null  int64  
 1   month      162049 non-null  int64  
 2   day        162049 non-null  int64  
 3   dep_time   161192 non-null  float64
 4   dep_delay  161192 non-null  float64
 5   arr_time   161061 non-null  float64
 6   arr_delay  160748 non-null  float64
 7   carrier    162049 non-null  object 
 8   tailnum    161801 non-null  object 
 9   flight     162049 non-null  int64  
 10  origin     162049 non-null  object 
 11  dest       162049 non-null  object 
 12  air_time   160748 non-null  float64
 13  distance   162049 non-null  int64  
 14  hour       161192 non-null  float64
 15  minute     161192 non-null  float64
dtypes: float64(7), int64(5), object(4)
memory usage: 21.0+ MB


As we will only be dealing with categorical features in this tutorial, it's better to filter them out. You can create a separate DataFrame consisting of only these features by running the following command. The method .copy() is used here so that any changes made in new DataFrame don't get reflected in the original one.


In [10]:
cat_df_flights = df_flights.select_dtypes(include=['object']) #chọn tất cả các cột có dtype là object
cat_df_flights.info() #check lại

<class 'pandas.core.frame.DataFrame'>
Index: 162049 entries, 0 to 162048
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   carrier  162049 non-null  object
 1   tailnum  161801 non-null  object
 2   origin   162049 non-null  object
 3   dest     162049 non-null  object
dtypes: object(4)
memory usage: 6.2+ MB


In [11]:
cat_df_flights

,carrier,tailnum,origin,dest
0,AS,N508AS,PDX,ANC
1,US,N195UW,SEA,CLT
2,UA,N37422,PDX,IAH
3,US,N547UW,PDX,CLT
4,AS,N762AS,SEA,ANC
...,...,...,...,...
162044,UA,N477UA,PDX,IAH
162045,DL,N137DL,SEA,ATL
162046,AS,N530AS,SEA,SAN
162047,US,NaN,PDX,PHX


In [12]:
cat_df_flights.isnull().sum()
#tailnum (NaN nhiều nhất)

,0
carrier,0
tailnum,248
origin,0
dest,0


In [13]:
cat_df_flights['tailnum'].nunique()
# mã số (có 3k) /  162048

3022

In [14]:
cat_df_flights['tailnum'].mode()[0] # Trả về giá trị xuất hiện nhiều nhất (mode) trong cột

'N223AG'

In [15]:
cat_df_flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162049 entries, 0 to 162048
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   carrier  162049 non-null  object
 1   tailnum  161801 non-null  object
 2   origin   162049 non-null  object
 3   dest     162049 non-null  object
dtypes: object(4)
memory usage: 6.2+ MB


## Fill missing data

In [16]:
# handle missing data by filling the most frequent appear in the columns

fill_tail = cat_df_flights['tailnum'].mode()[0]
print(fill_tail)
cat_df_flights = cat_df_flights.fillna(fill_tail)

N223AG


In [17]:
cat_df_flights.isnull().sum()
# completely handle NaN

,0
carrier,0
tailnum,0
origin,0
dest,0


In [18]:
cat_df_flights.head(5)

,carrier,tailnum,origin,dest
0,AS,N508AS,PDX,ANC
1,US,N195UW,SEA,CLT
2,UA,N37422,PDX,IAH
3,US,N547UW,PDX,CLT
4,AS,N762AS,SEA,ANC


## Frequency distribution of the diff categories in our dataset

In [19]:
cat_df_flights['carrier'].value_counts()

,count
carrier,
AS,62460
WN,23355
OO,18710
DL,16716
UA,16671
AA,7586
US,5946
B6,3540
VX,3272


In [20]:
cat_df_flights['carrier'].nunique()

11

In [21]:
cat_df_flights['carrier'].value_counts().count()

np.int64(11)

In [22]:
print('Total number of categories in Carrier Variable:',
      cat_df_flights['carrier'].value_counts().count())

Total number of categories in Carrier Variable: 11


# Encoding categorical data <a class="anchor" id="2"></a>

To keep it simple, you will apply these encoding methods only on the carrier column.
![image.png](attachment:image.png)

## Replace values or mapping

This is the most basic method, which is just replacing the categories with the desired numbers.

The idea is that you have the liberty to choose whatever numbers you want to assign to the categories according to the business use case.

In [23]:
cat_df_flights['carrier'].unique()

array(['AS', 'US', 'UA', 'DL', 'AA', 'F9', 'VX', 'OO', 'WN', 'B6', 'HA'],
      dtype=object)

In [24]:
{'AA': 1, 'AS': 2, 'B6': 3, 'DL': 4,
                                  'F9': 5, 'HA': 6, 'OO': 7 , 'UA': 8 ,
                           'US': 9,'VX': 10,'WN': 11}

{'AA': 1,
 'AS': 2,
 'B6': 3,
 'DL': 4,
 'F9': 5,
 'HA': 6,
 'OO': 7,
 'UA': 8,
 'US': 9,
 'VX': 10,
 'WN': 11}

In [53]:
# Step 1. Create a dictionary which contains mapping numbers for each category
replace_map = {'carrier': {'AA': 1, 'AS': 2, 'B6': 3, 'DL': 4,
                                  'F9': 5, 'HA': 6, 'OO': 7 , 'UA': 8 ,
                           'US': 9,'VX': 10,'WN': 11}}

In [54]:
replace_map2 = {'AA': 1000, 'AS': 2000, 'B6': 3000, 'DL': 4000,
                                  'F9': 5, 'HA': 6, 'OO': 7 , 'UA': 8 ,
                           'US': 9,'VX': 10,'WN': 11000}

In [55]:
cat_df_flights_replace = cat_df_flights.copy()
cat_df_flights_replace['carrier'].replace(replace_map2, inplace=True)

/tmp/ipython-input-963777001.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cat_df_flights_replace['carrier'].replace(replace_map2, inplace=True)
/tmp/ipython-input-963777001.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cat_df_flights_replace['carrier'].replace(replace_map2, inplace=True

In [56]:
cat_df_flights_replace

,carrier,tailnum,origin,dest
0,2000,N508AS,PDX,ANC
1,9,N195UW,SEA,CLT
2,8,N37422,PDX,IAH
3,9,N547UW,PDX,CLT
4,2000,N762AS,SEA,ANC
...,...,...,...,...
162044,8,N477UA,PDX,IAH
162045,4000,N137DL,SEA,ATL
162046,2000,N530AS,SEA,SAN
162047,9,N223AG,PDX,PHX


Note that defining a mapping via a hard coded dictionary is easy when the number of categories is low, like in this case which is 11.

In [57]:
cat_df_flights_replace = cat_df_flights.copy()

# Step 2. Replace the values with the corresponding map created
cat_df_flights_replace.replace(replace_map, inplace=True)

cat_df_flights_replace.head()

/tmp/ipython-input-555136850.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cat_df_flights_replace.replace(replace_map, inplace=True)


,carrier,tailnum,origin,dest
0,2,N508AS,PDX,ANC
1,9,N195UW,SEA,CLT
2,8,N37422,PDX,IAH
3,9,N547UW,PDX,CLT
4,2,N762AS,SEA,ANC


In [30]:
cat_df_flights.head()

,carrier,tailnum,origin,dest
0,AS,N508AS,PDX,ANC
1,US,N195UW,SEA,CLT
2,UA,N37422,PDX,IAH
3,US,N547UW,PDX,CLT
4,AS,N762AS,SEA,ANC


In [31]:
print('\nTypes of the variables for the dataset:\n',
      cat_df_flights_replace.dtypes)
# because we already assign the numbers so the type have changed from object to interger


Types of the variables for the dataset:
 carrier     int64
tailnum    object
origin     object
dest       object
dtype: object


***ENCODE WITH NUMBERS***

## Label Encoding

Numerical labels are always between 0 and n_categories-1.
![image.png](attachment:image.png)


In [58]:
# Create a bckup
cat_df_flights_le = cat_df_flights.copy()

In [59]:
from sklearn.preprocessing import LabelEncoder

# Step 1. Instantiate the model (label encoding)
lb_make = LabelEncoder()
lb_make

LabelEncoder()

In [34]:
cat_df_flights['carrier']

,carrier
0,AS
1,US
2,UA
3,US
4,AS
...,...
162044,UA
162045,DL
162046,AS
162047,US


In [35]:
lb_make.fit_transform(cat_df_flights['carrier'])
# use the column carrier in (cat_df_flights)

array([ 1,  8,  7, ...,  1,  8, 10])

In [36]:
#step1: lb_make = LabelEncoder()
#step2: data_label['col_code'] = lb_make.fit_transform(data['col'])

In [37]:
# Step 2. Fit the variable to the instatiated model
cat_df_flights_le['carrier_code'] = lb_make.fit_transform(cat_df_flights['carrier'])

cat_df_flights_le.head() #Results in appending a new column to df

#we assign based on the unqiue number that we have

,carrier,tailnum,origin,dest,carrier_code
0,AS,N508AS,PDX,ANC,1
1,US,N195UW,SEA,CLT,8
2,UA,N37422,PDX,IAH,7
3,US,N547UW,PDX,CLT,8
4,AS,N762AS,SEA,ANC,1


In [38]:
cat_df_flights_le.carrier_code.unique()

array([ 1,  8,  7,  3,  0,  4,  9,  6, 10,  2,  5])

In [39]:
cat_df_flights_le.carrier_code.value_counts()

,count
carrier_code,
1,62460
10,23355
6,18710
3,16716
7,16671
0,7586
8,5946
2,3540
9,3272


In [40]:
cat_df_flights_le.carrier.value_counts()

,count
carrier,
AS,62460
WN,23355
OO,18710
DL,16716
UA,16671
AA,7586
US,5946
B6,3540
VX,3272


    Though label encoding is straight but it has the disadvantage that the numeric values can be misinterpreted by algorithms as having some sort of hierarchy/order in them.

## Ordinal Encoder

Same as `LabelEncoder` but you can apply the encoding to several categorical variables



In [41]:
# Create a backup
cat_df_flights_oe = cat_df_flights.copy()

In [42]:
!pip install category-encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.2 MB/s eta 0:00:00


In [43]:
import category_encoders as ce

# Step 1. Instantiate the object with the necessary inputs
encoder = ce.OrdinalEncoder(cols=['carrier', 'origin', 'tailnum'])
encoder

OrdinalEncoder(cols=['carrier', 'origin', 'tailnum'])

In [44]:
# Step 2. Fit the variable to the instatiated model
cat_df_flights_oe = encoder.fit_transform(cat_df_flights_oe)

In [45]:
cat_df_flights_oe.head(5)

,carrier,tailnum,origin,dest
0,1,1,1,ANC
1,2,2,2,CLT
2,3,3,1,IAH
3,2,4,1,CLT
4,1,5,2,ANC


In [46]:
cat_df_flights.origin.unique()

array(['PDX', 'SEA'], dtype=object)

In [47]:
cat_df_flights_oe.origin.unique()

array([1, 2])

In [48]:
cat_df_flights.carrier#.unique()

,carrier
0,AS
1,US
2,UA
3,US
4,AS
...,...
162044,UA
162045,DL
162046,AS
162047,US


In [49]:
cat_df_flights_oe.carrier#.unique()

,carrier
0,1
1,2
2,3
3,2
4,1
...,...
162044,3
162045,4
162046,1
162047,2


## One-Hot encoding

The basic strategy is to convert each category value into a new column and assign a 1 or 0 (True/False) value to the column. This has the benefit of not weighting a value improperly

<img src="https://miro.medium.com/max/3758/1*O_pTwOZZLYZabRjw3Ga21A.png" width="500">

In [60]:
cat_df_flights_onehot = cat_df_flights.copy()

In [67]:
# 11 columns if we use one-shot
cat_df_flights['carrier'].unique()

array(['AS', 'US', 'UA', 'DL', 'AA', 'F9', 'VX', 'OO', 'WN', 'B6', 'HA'],
      dtype=object)

In [62]:
cat_df_flights_onehot = pd.get_dummies(cat_df_flights_oh,
                                       columns=['carrier'])

# split 1 columns into many columns with the name in each columns is the unqunie values

In [63]:
cat_df_flights_onehot.head(3)

,tailnum,origin,dest,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_HA,carrier_OO,carrier_UA,carrier_US,carrier_VX,carrier_WN
0,N508AS,PDX,ANC,False,True,False,False,False,False,False,False,False,False,False
1,N195UW,SEA,CLT,False,False,False,False,False,False,False,False,True,False,False
2,N37422,PDX,IAH,False,False,False,False,False,False,False,True,False,False,False


In [64]:
cat_df_flights_onehot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162049 entries, 0 to 162048
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   tailnum     162049 non-null  object
 1   origin      162049 non-null  object
 2   dest        162049 non-null  object
 3   carrier_AA  162049 non-null  bool  
 4   carrier_AS  162049 non-null  bool  
 5   carrier_B6  162049 non-null  bool  
 6   carrier_DL  162049 non-null  bool  
 7   carrier_F9  162049 non-null  bool  
 8   carrier_HA  162049 non-null  bool  
 9   carrier_OO  162049 non-null  bool  
 10  carrier_UA  162049 non-null  bool  
 11  carrier_US  162049 non-null  bool  
 12  carrier_VX  162049 non-null  bool  
 13  carrier_WN  162049 non-null  bool  
dtypes: bool(11), object(3)
memory usage: 6.6+ MB


In [68]:
cat_df_flights_onehot.filter(regex = 'carrier')

,carrier_AA,carrier_AS,carrier_B6,carrier_DL,carrier_F9,carrier_HA,carrier_OO,carrier_UA,carrier_US,carrier_VX,carrier_WN
0,False,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False
4,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
162044,False,False,False,False,False,False,False,True,False,False,False
162045,False,False,False,True,False,False,False,False,False,False,False
162046,False,True,False,False,False,False,False,False,False,False,False
162047,False,False,False,False,False,False,False,False,True,False,False


In [69]:
print('\n------------- Raw dataset----------------\n')
print('Shape of the raw dataset',cat_df_flights.shape)
#print('Unique values of the raw dataset\n',cat_df_flights.carrier.value_counts())
print('First values of the raw dataset',cat_df_flights.head(2))

print('\n------------- Encoded dataset----------------\n')
print('Shape of the encoded dataset',cat_df_flights_onehot.shape)
print('First values of the encoded dataset',cat_df_flights_onehot.head(2))



------------- Raw dataset----------------

Shape of the raw dataset (162049, 4)
First values of the raw dataset   carrier tailnum origin dest
0      AS  N508AS    PDX  ANC
1      US  N195UW    SEA  CLT

------------- Encoded dataset----------------

Shape of the encoded dataset (162049, 14)
First values of the encoded dataset   tailnum origin dest  carrier_AA  carrier_AS  carrier_B6  carrier_DL  carrier_F9  carrier_HA  carrier_OO  carrier_UA  carrier_US  carrier_VX  carrier_WN
0  N508AS    PDX  ANC       False        True       False       False       False       False       False       False       False       False       False
1  N195UW    SEA  CLT       False       False       False       False       False       False       False       False        True       False       False


Số cột tăng từ 4 → 14 là logic
→ 1 cột carrier → ~11 cột carrier_

Đây là One-Hot Encoding đúng về mặt kỹ thuật
✔ Phù hợp cho:

Logistic Regression

Linear / Ridge / Lasso

Tree-based models


**While one-hot encoding solves the problem of unequal weights given to categories within a feature, it is not very useful when there are many categories, as that will result in formation of as many new columns, which can result in the curse of dimensionality.**

## Binary encoding

In this technique, first the categories are encoded as ordinal, then those integers are converted into binary code, then the digits from that binary string are split into separate columns. This encodes the data in fewer dimensions than one-hot.

Binary encoding works like this:
- The categories are encoded as ordinal, for example, categories like red, yellow, green are assigned labels as 1, 2, 3 (let’s assume).
- These integers are then converted into binary code, so for example 1 becomes 001 and 2 becomes 010 and so on.
- Then the digits from that binary string are split into separate columns.

<img src="https://miro.medium.com/max/6434/1*VuNZWUX6b7GUGB0zRu2zrA.png" width="800">

http://decimal-to-binary.com/decimal-to-binary-converter-online.html?id=1080

**BINARY= NUMBER
ONESHOT = TRUE/FALSE**

In [70]:
cat_df_flights_ce = cat_df_flights.copy()

In [71]:
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['carrier'])

df_binary = encoder.fit_transform(cat_df_flights_ce)

df_binary.head()



,carrier_0,carrier_1,carrier_2,carrier_3,tailnum,origin,dest
0,0,0,0,1,N508AS,PDX,ANC
1,0,0,1,0,N195UW,SEA,CLT
2,0,0,1,1,N37422,PDX,IAH
3,0,0,1,0,N547UW,PDX,CLT
4,0,0,0,1,N762AS,SEA,ANC


In [72]:
print('ONE-HOT -Encoded variables \n',
      cat_df_flights_onehot.filter(regex= 'carrier').head(5))

print('\nBinary encoding - Encoded variables \n',
      df_binary.filter(regex= 'carrier').head(5))

ONE-HOT -Encoded variables 
    carrier_AA  carrier_AS  carrier_B6  carrier_DL  carrier_F9  carrier_HA  carrier_OO  carrier_UA  carrier_US  carrier_VX  carrier_WN
0       False        True       False       False       False       False       False       False       False       False       False
1       False       False       False       False       False       False       False       False        True       False       False
2       False       False       False       False       False       False       False        True       False       False       False
3       False       False       False       False       False       False       False       False        True       False       False
4       False        True       False       False       False       False       False       False       False       False       False

Binary encoding - Encoded variables 
    carrier_0  carrier_1  carrier_2  carrier_3
0          0          0          0          1
1          0          0          1  